In [1]:
import os
import torch
import cv2 as cv
import numpy as np
import torchvision
from torchvision import transforms
import pandas as pd

In [2]:
# case can have to be mix of this two groops (rgb, dept, Ergbd, Lrgbd) and (rad, angle)

case = 'Prgbd_rad'
ver = 1

rgb_ver = 1

In [3]:
rgb_img_dir = './dataset/test/rgb/'
depth_img_dir = './dataset/test/depth/'

csv_path = './dataset/test/test_data.csv'

In [4]:
# Loads model.
def model_definition(rgb_weights, depth_weights, classes):
    
    rgb_model = torchvision.models.resnet18(pretrained=False)
    rgb_model.fc = torch.nn.Linear(512, classes)
    
    depth_model = torchvision.models.resnet18(pretrained=False)
    depth_model.fc = torch.nn.Linear(512, classes)
    depth_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    rgb_model.eval()
    depth_model.eval()
    
    rgb_model.load_state_dict(torch.load(rgb_weights)) # Load weights
    depth_model.load_state_dict(torch.load(depth_weights)) # Load weights

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    rgb_model.to(device)
    depth_model.to(device)

    return rgb_model, depth_model, device

def mask (rgb, depth):
    
    depth = cv.cvtColor(depth, cv.COLOR_BGR2GRAY)
    _, mask = cv.threshold(depth, 100, 255, cv.THRESH_BINARY)
    kernel = np.ones((9, 9), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    masked_rgb = cv.bitwise_and(rgb, rgb, mask = mask.astype('uint8'))

    return masked_rgb

# transfer image to tensor.
def im2tens(rgb, depth):
        
        rgb_transform = torchvision.transforms.ToTensor()
        depth_transform = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Grayscale(num_output_channels=1)])
                 
        rgb_tensor = rgb_transform(rgb)
        depth_tensor = depth_transform(depth)

        return rgb_tensor, depth_tensor


# Pregict class.
def predict(img, model, device):
    
    xb = img.unsqueeze(0)
    yb = model(xb.to(device))
    # Get the normalized probability vector
    p = torch.nn.functional.softmax(yb, dim=1)
    dist = p[0].cpu().detach().numpy()
    
    return dist
            

def get_models(rgb_angle_weights, depth_angle_weights, rgb_rad_weights, depth_rad_weights):

    rgb_angle_model, depth_angle_model, device = model_definition(rgb_angle_weights, depth_angle_weights, 16)
    rgb_rad_model, depth_rad_model, _ = model_definition(rgb_rad_weights, depth_rad_weights, 6)

    return rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device

# Calculates an average between the predicted vector and the values of the angles of the middle of the slices.
def angle_avg_pred(pred, th=0.01):
    
    pred = np.where(pred < th, 0, pred)
    pred = pred/np.sum(pred)
    step = 2*np.pi / len(pred)
    mid_slice = [step/2 + step * interval for interval in range(len(pred))]
    
    sin = np.zeros((len(mid_slice)))
    cos = np.zeros((len(mid_slice)))
    for ii in range(len(mid_slice)):
        sin[ii] = np.sin(mid_slice[ii])
        cos[ii] = np.cos(mid_slice[ii])
        
    dot_sin = np.dot(sin, pred)
    dot_cos = np.dot(cos, pred)
    tan = np.arctan2(dot_sin ,dot_cos)
    avg_pred = np.rad2deg(tan)
    if avg_pred < 0: avg_pred = avg_pred+360
        
    return avg_pred

def rad_avg_pred(pred, th=0.05):
    
    pred = np.where(pred < th, 0, pred)
    pred = pred/np.sum(pred)
    step = np.linspace(0, 6, 6)
    avg_pred = np.dot(step, pred) 
        
    return avg_pred

def main_pred(rgb_img, depth_img, rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device):
    
    masked_rgb = mask (rgb_img, depth_img)
    rgb_tensor, depth_tensor = im2tens(masked_rgb, depth_img)
    
    rgb_angle_dist = predict(rgb_tensor, rgb_angle_model, device)
    depth_angle_dist = predict(depth_tensor, depth_angle_model, device)
    rgb_rad_dist = predict(rgb_tensor, rgb_rad_model, device)
    depth_rad_dist = predict(depth_tensor, depth_rad_model, device)
    
    angle_dist = (rgb_angle_dist+depth_angle_dist)/2
    rad_dist = (rgb_rad_dist+depth_rad_dist)/2
    
    angle_pred = angle_avg_pred(angle_dist)
    rad_pred = rad_avg_pred(rad_dist)

    
    return rad_pred, angle_pred

In [5]:
rgb_angle_weights = './Weights/rgb_angle_v' + str(rgb_ver) + '.pth'
depth_angle_weights = './Weights/depth_angle_v' + str(rgb_ver) + '.pth'
rgb_rad_weights  = './Weights/rgb_rad_v' + str(rgb_ver) + '.pth'
depth_rad_weights  = './Weights/depth_rad_v' + str(rgb_ver) + '.pth'

rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device = get_models(rgb_angle_weights, depth_angle_weights, rgb_rad_weights, depth_rad_weights)

In [6]:
output_dir = './results/' + case.split('_')[0] + '_v' + str(ver) + '/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_csv_path = os.path.join(output_dir, 'graphs_data.csv')

df = pd.read_csv(csv_path)
im_name = df.loc[:, 'Picture name']
angle = df.loc[:, 'Angle [deg]']
rad = df.loc[:, 'Radial distance [mm]']

data = []
        
for ii in range(len(im_name)):
    
    rgb_img = cv.cvtColor(cv.imread(rgb_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    depth_img = cv.cvtColor(cv.imread(depth_img_dir + im_name[ii]), cv.COLOR_BGR2RGB)
    rad_pred, angle_pred = main_pred(rgb_img, depth_img, rgb_angle_model, depth_angle_model, rgb_rad_model, depth_rad_model, device)    
         
    if np.abs(angle_pred - angle[ii]) > 180:
        angle_pred = 360 - angle_pred
        
    data.append({
        'Picture name': im_name[ii],
        'gt_rad': rad[ii],
        'pred_rad': rad_pred,
        'gt_angle': angle[ii],
        'pred_angle': angle_pred
    })
    
# Convert the list of dictionaries to a DataFrame
data_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
data_df.to_csv(output_csv_path, index=False)